# 자율주행 모델 데모 v.1
---
> 영상 데이터만을 이용해 1 차적으로 주행판단을 하는 데모. 방식은 `지도학습`

* label 은 좌, 우, 전진의 3 가지 action 을 classification 하는 분류식


## 과정
---
1. label 생성 : `원본` - 프레임마다 `키보드로 라벨 기록` (0 초 ~ x 초 까지)
2. 영상학습 과정 : `원본` - `전처리` - `CNN` - `출력과 라벨 비교` (0 초 ~ x 초 까지)
3. 검증 과정 : `새로운 영상` - `전처리` - `CNN` - `출력확인`

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import tensorflow as tf
import pickle
import copy

## Process 1 - label 생성
* reduced shape = (h : 360, w : 640, ch : 3)

In [2]:
cap = cv2.VideoCapture('test.mp4')
label_data = []
cnt_frame = 0
command = [0,1,0]
com_str = str()

while(cap.isOpened()):
    ret, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR) # 이미지 size 1/4 로 축소
    
    k = cv2.waitKey(1)
    if(k == ord('q')):
        break
    elif(k == ord('a')):
        command = [1,0,0]
        com_str = 'left'
    elif(k == ord('d')):
        command = [0,0,1]
        com_str = 'right'
    elif(k == ord('w')):
        command = [0,1,0]
        com_str = 'straight'
    
    label_data.append(command)
    
    cv2.putText(frame, com_str, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .7, (0, 0, 0), 2)
    cv2.imshow('frame', frame)    
    cnt_frame += 1

with open('label_data.txt', 'w') as f:
    f.write(str(label_data))
    
print('frame 카운트 :', cnt_frame, '\nlable 카운트 :', len(label_data), label_data)
cap.release()
cv2.destroyAllWindows()

frame 카운트 : 346 
lable 카운트 : 346 [[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]

## Process 2 - 영상학습과정

In [5]:
class CNN:
    def __init__(self, h, w, sess):
        self.size_h = h
        self.size_w = w
        self.sess = sess
        self.model = self.make_model()
    
    def make_model(self):
        # 모델
        self.input_data = tf.placeholder(shape=[None, self.size_h, self.size_w, 3], dtype=tf.float32) # 이미지 데이터
        self.label = tf.placeholder(shape=[3], dtype=tf.int16)
        
        self.w_in = tf.Variable(tf.random_normal([5,5,3,8], stddev=.001)) # conv 1 가중치
        self.l1 = tf.nn.conv2d(self.input_data, self.w_in, strides=[1,1,1,1], padding='SAME')
        self.l1 = tf.nn.relu(self.l1)
        self.l1 = tf.nn.max_pool(self.l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
        self.w_out = tf.Variable(tf.random_normal(shape=[self.size_w//2*self.size_h//2*8, 3], stddev=.001))
        self.b = tf.Variable(tf.random_normal([3]))
        self.h_flat = tf.reshape(self.l1, [-1, self.size_w//2*self.size_h//2*8])
        
        self.output = tf.matmul(self.h_flat, self.w_out) + self.b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.label))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(self.cost)
    
    def train(self, img, label):
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict={self.input_data:img.reshape([1,360,640,3]), self.label:label})
        print(cost)

In [8]:
# 학습 과정
label_train = copy.deepcopy(label_data)

with tf.Session() as sess:
    cap = cv2.VideoCapture('test.mp4')
    model = CNN(360, 640, sess)
    initializer = tf.global_variables_initializer()
    sess.run(initializer)

    while(cap.isOpened()):
        ret, frame = cap.read()
        frame = cv2.resize(frame, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR) # 이미지 size 1/4 로 축소    
        #print(frame.shape)
        model.train(frame, label_train.pop(0))
        if(not ret):
            break

0.05937419
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [ ]:
cap = cv2.VideoCapture(best.url)
while(True):
    ret, frame = cap.read()
    if(not ret):
        break
    
    # 영상정의
    sliced = frame[:, :320, :]
    gray = cv2.cvtColor(sliced, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    blur = cv2.GaussianBlur(binary, (15, 15), 0)
    dilation = cv2.dilate(blur, cv2.getStructuringElement(cv2.MORPH_RECT,(10, 10)), iterations = 1)
    canny = cv2.Canny(dilation, 30, 70)
    
    cv2.imshow('frame', sliced)
    #cv2.imshow('gray', gray)
    #cv2.imshow('bin', binary)
    #cv2.imshow('blur', blur)
    cv2.imshow('dilate', dilation)
    cv2.imshow('canny', canny)
    
    key = cv2.waitKey(25)
    if key == 27:   #Esc누르면 종료. x키 열심히 눌러도 소용없음. Esc를 눌러야함.
            break
cv2.destroyAllWindows()